In [35]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
sns.set_context('paper')
# pd.set_option('display.max_columns', None)

In [36]:
# association recognition - cued recall
with open("../Data/simuS2_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
# df_study = df_study.loc[df_study.session < 300]
# df_test = df_test.loc[df_test.session < 300]

In [37]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,pair_idx,list,session,subject
0,1026,897,PATRIOT,MINK,14,0,0,0
1,1524,1231,TUTU,SANDPAPER,0,0,0,0
2,893,437,MILDEW,DANCER,1,0,0,0
3,1492,1034,TOY,PEARL,5,0,0,0
4,1556,1586,VENT,WAND,21,0,0,0
...,...,...,...,...,...,...,...,...
43195,34,968,APPLICATION,OATS,72,2,99,99
43196,1165,250,RAM,CART,71,2,99,99
43197,600,1144,FRAME,PUPPY,60,2,99,99
43198,345,1086,COLLAR,PLATE,73,2,99,99


In [38]:
df_test

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,pair_idx,type,test,list,session,subject
0,103,-1,BEAD,None,0,-1,extra,1,0,0,0
1,809,1001,LEG,PAGE,1,16,Intact_Pair,1,0,0,0
2,51,-1,ASTRONAUT,None,1,13,Item_Pair,1,0,0,0
3,1512,-1,TRUCK,None,0,-1,extra,1,0,0,0
4,1026,-1,PATRIOT,None,1,14,Item_Pair,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
35995,734,1328,IGLOO,SLOPE,0,87,Intact_Pair,2,2,99,99
35996,886,-1,MEDICINE,None,0,-1,extra,2,2,99,99
35997,1088,-1,PLAZA,None,0,-1,extra,2,2,99,99
35998,939,-1,NECK,None,1,69,Same_Item,2,2,99,99


### Run CMR-IA

In [39]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [40]:
params = cmr.make_default_params()
params.update(learn_while_retrieving = True)

In [41]:
# params.update(
#     beta_enc = 0.4,
#     beta_rec = 0.1, # beta for retrieved item
#     beta_cue = 0.3,
#     beta_rec_post = 0.9,
#     beta_distract = 0.3,
#     gamma_fc = 0.03,
#     s_fc = 0.3,
#     c_thresh_itm = 0.30,
#     c_thresh_ass = 0.50,
#     learn_while_retrieving = True,
#     d_ass = 0.3,
# )
# params

In [42]:
# pso results
import sys
sys.path.append("../Modeling/CMR_IA/fitting")
from optimization_utils import make_boundary

_,_,what_to_fit = make_boundary(sim_name='S2')
optim_params = np.loadtxt("../Modeling/CMR_IA/fitting/outparams_backup/S2_230807_100-200.txt")
for pname, pvalue in zip(what_to_fit, optim_params):
    params[pname] = pvalue
params

{'beta_enc': 0.499778,
 'beta_rec': 0.114594,
 'beta_cue': 0.101183,
 'beta_rec_post': 0.286028,
 'beta_distract': 0.152172,
 'phi_s': 2,
 'phi_d': 0.5,
 's_cf': 0,
 's_fc': 0.452008,
 'kappa': 0.5,
 'eta': 0.5,
 'omega': 5,
 'alpha': 1,
 'c_thresh': 0.5,
 'c_thresh_itm': 0.90571,
 'c_thresh_ass': 0.753459,
 'd_ass': 0.37493,
 'lamb': 0.5,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'thresh_sigma': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.984545,
 'gamma_cf': 0.5}

In [43]:
params.update(thresh_sigma = 0.2)

In [44]:
df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params, df_study, df_test, s_mat, mode="Recog-Recog")
df_simu['test'] = df_test['test']
df_simu = df_simu.merge(df_test,on=['session','list','test','test_itemno1','test_itemno2'])
df_simu

CMR2 Time: 109.08472275733948


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,pair_idx,type,subject
0,0,0,103,-1,0.0,115.412285,0.746267,1,BEAD,None,0,-1,extra,0
1,0,0,809,1001,1.0,1246.127197,0.946189,1,LEG,PAGE,1,16,Intact_Pair,0
2,0,0,51,-1,1.0,2027.063965,0.921862,1,ASTRONAUT,None,1,13,Item_Pair,0
3,0,0,1512,-1,0.0,13.844074,0.640234,1,TRUCK,None,0,-1,extra,0
4,0,0,1026,-1,1.0,1869.088867,0.925918,1,PATRIOT,None,1,14,Item_Pair,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,99,2,734,1328,1.0,29.738016,0.678463,2,IGLOO,SLOPE,0,87,Intact_Pair,99
35996,99,2,886,-1,0.0,1.508578,0.529400,2,MEDICINE,None,0,-1,extra,99
35997,99,2,1088,-1,0.0,3.099023,0.565396,2,PLAZA,None,0,-1,extra,99
35998,99,2,939,-1,1.0,2318.251709,0.915150,2,NECK,None,1,69,Same_Item,99


In [45]:
# get correctness and condition
df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans
def get_cond(x):
    this_type = x['type']
    target = x['correct_ans']
    if target == 1:
        if this_type == "Different_Item":
            return "Different_Item"
        elif this_type == "Item_Pair":
            return "Item_Pair"
        elif this_type == "Pair_Item":
            return "Pair_Item"
        elif this_type == "Same_Item":
            return "Same_Item"
        elif this_type == "Intact_Pair":
            return "Intact_Pair"
    elif target == 0:
        if this_type == "extra":
            return "NR_Lure"
        elif this_type == "Same_Item" or this_type == "Intact_Pair":
            return "Repeated_Lure"
        else:
            return "Discard"
df_simu['condition'] = df_simu.apply(get_cond,axis=1)
df_simu

,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,pair_idx,type,subject,correct,condition
0,0,0,103,-1,0.0,115.412285,0.746267,1,BEAD,None,0,-1,extra,0,True,NR_Lure
1,0,0,809,1001,1.0,1246.127197,0.946189,1,LEG,PAGE,1,16,Intact_Pair,0,True,Intact_Pair
2,0,0,51,-1,1.0,2027.063965,0.921862,1,ASTRONAUT,None,1,13,Item_Pair,0,True,Item_Pair
3,0,0,1512,-1,0.0,13.844074,0.640234,1,TRUCK,None,0,-1,extra,0,True,NR_Lure
4,0,0,1026,-1,1.0,1869.088867,0.925918,1,PATRIOT,None,1,14,Item_Pair,0,True,Item_Pair
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,99,2,734,1328,1.0,29.738016,0.678463,2,IGLOO,SLOPE,0,87,Intact_Pair,99,False,Repeated_Lure
35996,99,2,886,-1,0.0,1.508578,0.529400,2,MEDICINE,None,0,-1,extra,99,True,NR_Lure
35997,99,2,1088,-1,0.0,3.099023,0.565396,2,PLAZA,None,0,-1,extra,99,True,NR_Lure
35998,99,2,939,-1,1.0,2318.251709,0.915150,2,NECK,None,1,69,Same_Item,99,True,Same_Item


In [46]:
# # get pairs and optimal csim
# df_simu['pair'] = df_simu.apply(lambda x: False if x['test_itemno2'] == -1 else True, axis=1)

# csim_two = df_simu.groupby(["pair","correct_ans"]).csim.mean()
# print(csim_two)

# opt_thresh_item = np.mean(csim_two[False])
# opt_thresh_pair = np.mean(csim_two[True])
# print(opt_thresh_item)
# print(opt_thresh_pair)

# def adjust_resp(x):
#     ispair = x['pair']
#     csim = x['csim']
#     if ispair:
#         if csim > opt_thresh_pair:
#             return 1
#         else:
#             return 0
#     else:
#         if csim > opt_thresh_item:
#             return 1
#         else:
#             return 0
# df_simu['s_resp'] = df_simu.apply(adjust_resp, axis=1)
# df_simu

### Analysis

In [47]:
# anal_perform used by pso
import sys
sys.path.append("../Modeling/CMR_IA/fitting")
from object_funcs import anal_perform_S2

In [48]:
# def anal_perform_S2(df_simu):

#     # get target items
#     df_target = df_simu.query("condition != 'Discard'")

#     # get pairs data
#     def get_pair(df_tmp):
#         df_tmp_pair = pd.pivot_table(df_tmp,index=["pair_idx","condition"],columns="test",values="correct")
#         df_tmp_pair.columns = ["test1","test2"]
#         df_tmp_pair.reset_index(inplace=True)
#         return df_tmp_pair

#     df_p = df_target.query("condition != 'NR_Lure'")
#     df_pair = get_pair(df_p).reset_index()

#     # get Q values
#     def Yule_Q(A, B, C, D):
#         return (A * D - B * C) / (A * D + B * C)
    
#     qs = []
#     conditions = ['Different_Item', 'Item_Pair', 'Pair_Item', 'Intact_Pair', 'Same_Item', 'Repeated_Lure', 'NR_Lure']
#     for cond in conditions:
#         df_tmp = df_pair.query(f"condition == '{cond}'")
#         test2_rsp = pd.Categorical(df_tmp.test2, categories=[0,1])
#         test1_rsp = pd.Categorical(df_tmp.test1, categories=[0,1])
#         df_tab = pd.crosstab(index=test2_rsp,columns=test1_rsp, rownames=['test2'], colnames=['test1'], normalize=False, dropna=False)

#         try:
#             q = Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5)
#         except:
#             q = 0 if cond == 'NR_Lure' else np.nan

#         qs.append(q)
    
#     # get hit rates and aggregate
#     df_res = pd.DataFrame({'Condition':conditions, 'Q':qs})
#     df_res.set_index('Condition', inplace=True)
#     df_res['Test1_p'] = df_target.groupby(["test","condition"])["s_resp"].mean()[1]
#     df_res['Test2_p'] = df_target.groupby(["test","condition"])["s_resp"].mean()[2]
#     df_res = df_res[['Test1_p','Test2_p','Q']]
#     stats = df_res.values.tolist()

#     return stats

In [49]:
subjects = np.unique(df_simu.subject)
stats = []
for subj in subjects:
    df_subj = df_simu.query(f"subject=={subj} and list % 3 != 0") # discard first list
    # df_subj = df_simu.query(f"subject=={subj}")
    stats_subj = anal_perform_S2(df_subj)
    stats.append(stats_subj)
stats_mean = np.nanmean(stats, axis=0)
stats_mean

array([[ 0.61490332,  0.47763384, -0.01374987],
       [ 0.58578144,  0.87753439,  0.32309048],
       [ 0.91660931,  0.47383369,  0.03902189],
       [ 0.90083333,  0.85266667,  0.53738789],
       [ 0.55471717,  0.50099206,  0.09786403],
       [ 0.13583702,  0.15422189,  0.63342251],
       [ 0.1615    ,  0.13266667,  0.        ]])

In [50]:
ground_truth = np.array([[0.82, 0.68, 0.26], 
                         [0.82, 0.85, 0.64],
                         [0.91, 0.85, 0.59],
                         [0.81, 0.82, 0.86],
                         [0.90, 0.92, 0.94],
                         [0.07, 0.15, 0.54],
                         [0.07, 0.06, 0]])
err = np.mean(np.power(stats_mean-ground_truth,2))
err

0.09063026342705102

In [51]:
import scipy as sp
print(sp.stats.sem(np.array(stats),axis=0, nan_policy='omit').round(3))

[[0.015 0.014 0.049]
 [0.01 0.008 0.046]
 [0.009 0.016 0.047]
 [0.013 0.015 0.039]
 [0.014 0.016 0.046]
 [0.021 0.022 0.033]
 [0.004 0.004 0.0]]
